In [1]:
import os
from pybullet_suite import *
import open3d as o3d
import trimesh

import yaml
import tqdm
import pandas as pd
from IPython.display import clear_output

from sdf_world.grasp_sim import * 
from sdf_world.robots import HANDE_URDF
from pathlib import Path

pybullet build time: May 20 2022 19:44:17


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# generate world
world = BulletWorld(gui=True, gravity=[0,0,0])
sm = BulletSceneMaker(world)
clear_output(wait=True)

ven = NVIDIA Corporation


ven = NVIDIA Corporation


In [3]:
# hand
tcp_offset_pose = Pose(rot=Rotation.from_euler("zyx",[np.pi/2, 0,0]), trans=[0,0,0.09])
hand = Gripper("hand", HANDE_URDF, tcp_offset_pose, 0.05,
        world, sm, finger_swept_vol=True, joint_rev=True)

In [4]:
#load obj_mesh
obj_name = "waffle_box"
obj_folder_path = Path("./sdf_world/assets/"+obj_name)
obj_mesh_path = obj_folder_path / (obj_name + ".obj")
#obj_mesh_path = obj_folder_path / ("cup_vhacd" + ".obj")

In [5]:
obj_mesh = trimesh.load(obj_mesh_path)

In [6]:
# normalizing
sphere_radius = 0.8
vertices = obj_mesh.vertices - obj_mesh.bounding_box.centroid #centered
scale_to_norm = sphere_radius * 2 / np.max(obj_mesh.bounding_box.extents)
#vertices *= scale_to_norm
#obj_mesh_normalized = trimesh.Trimesh(vertices, obj_mesh.faces, obj_mesh.face_normals, obj_mesh.vertex_normals)
obj_mesh_centered = trimesh.Trimesh(vertices, obj_mesh.faces, obj_mesh.face_normals, obj_mesh.vertex_normals)

# export normalized mesh
# suffix = f"_scale{np.round(scale_to_norm,2)}.obj"
# obj_mesh_norm_path = obj_mesh_path.as_posix().replace(".obj", suffix)
# obj_mesh_normalized.export(obj_mesh_norm_path, file_type="obj")
obj_mesh_centered_path = obj_folder_path/(obj_name+"_centered.obj")
obj_mesh_centered.export(obj_mesh_centered_path, file_type="obj")

'# https://github.com/mikedh/trimesh\nv 0.06650000 -0.01075000 0.03500000\nv 0.06650000 -0.01075000 0.03500000\nv 0.06650000 -0.01075000 0.03500000\nv 0.06650000 -0.01075000 -0.03500000\nv 0.06650000 -0.01075000 -0.03500000\nv 0.06650000 -0.01075000 -0.03500000\nv 0.06650000 0.01075000 -0.03500000\nv 0.06650000 0.01075000 -0.03500000\nv 0.06650000 0.01075000 -0.03500000\nv 0.06650000 0.01075000 0.03500000\nv 0.06650000 0.01075000 0.03500000\nv 0.06650000 0.01075000 0.03500000\nv -0.06650000 0.01075000 -0.03500000\nv -0.06650000 0.01075000 -0.03500000\nv -0.06650000 0.01075000 -0.03500000\nv -0.06650000 0.01075000 0.03500000\nv -0.06650000 0.01075000 0.03500000\nv -0.06650000 0.01075000 0.03500000\nv -0.06650000 -0.01075000 -0.03500000\nv -0.06650000 -0.01075000 -0.03500000\nv -0.06650000 -0.01075000 -0.03500000\nv -0.06650000 -0.01075000 0.03500000\nv -0.06650000 -0.01075000 0.03500000\nv -0.06650000 -0.01075000 0.03500000\nvn 1.00000000 0.00000000 0.00000000\nvn 0.00000000 -1.00000000

In [7]:
# vhacd
world.physics_client.vhacd(
    obj_mesh_path.as_posix(), 
    (obj_folder_path/"cup_vhacd.obj").as_posix(), 
    "log.txt")

V-HACD V2.2
Syntax: testVHACD [options] --input infile.obj --output outfile.obj --log logfile.txt

Options:
       --input                     Wavefront .obj input file name
       --output                    VRML 2.0 output file name
       --log                       Log file name
       --resolution                Maximum number of voxels generated during the voxelization stage (default=100,000, range=10,000-16,000,000)
       --depth                     Maximum number of clipping stages. During each split stage, parts with a concavity higher than the user defined threshold are clipped according the "best" clipping plane (default=20, range=1-32)
       --concavity                 Maximum allowed concavity (default=0.0025, range=0.0-1.0)
       --planeDownsampling         Controls the granularity of the search for the "best" clipping plane (default=4, range=1-16)
       --convexhullDownsampling    Controls the precision of the convex-hull generation process during the clipping plane 

In [7]:
#obj in pybullet
obj = world.load_mesh(
    "obj", obj_mesh_centered_path.as_posix(), obj_mesh_centered_path.as_posix(), Pose.identity())

In [8]:
sim = GraspSim(hand, obj, obj_mesh_centered)

In [9]:
df_succ, df_fail = sim.execute(
    visualize=True, surface_points=10000, grasp_samples=100)

Antipodal grasp sampling...


  0%|          | 0/100 [00:00<?, ?it/s]

 58%|█████▊    | 58/100 [00:21<00:19,  2.20it/s]

 59%|█████▉    | 59/100 [00:21<00:19,  2.11it/s]

In [22]:
def to_pointcloud(xyz):
    return o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyz))

In [23]:
xyz_label = ["x", "y", "z"]
xyz_fail = df_succ.loc[:,xyz_label].to_numpy()
frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1)
o3d.visualization.draw_geometries([to_pointcloud(xyz_fail), frame])

In [24]:
df_succ_scaled = df_succ.copy()
df_fail_scaled = df_fail.copy()
df_succ.loc[:,xyz_label] *= scale_to_norm
df_fail.loc[:,xyz_label] *= scale_to_norm

# save dataframes
data_succ_path = obj_folder_path / "succ.csv"
data_fail_path = obj_folder_path / "fail.csv"
df_succ_scaled.to_csv(data_succ_path, index=False)
df_fail_scaled.to_csv(data_fail_path, index=False)


# save info
import pickle
obj_info_path = obj_folder_path / "info.pkl"
obj_info = {'name':"waffle_box", "scale_to_norm":scale_to_norm}
with open(obj_info_path, "wb") as f:
    pickle.dump(obj_info, f, protocol=pickle.HIGHEST_PROTOCOL)

PosixPath('sdf_world/assets/waffle_box/info.pkl')